# Machine learning model LaQuiniela

In this notebook we are going to develop a machine learning model to predict, given the season, the division and the matchday, the result of each match, given the season, the division and the matchday.

The library `sqlite3` is for reading the files `.sqlite`, in particular, `laliga.sqlite`, with the information of the matches, and `clasification.sqlite`, where we have store the data from exercise 10.

The library `sklearn` is for machine learning. 

In [141]:
import pandas as pd
import numpy as np
import sqlite3

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

Leo los datos usando la libreia `sqlite3`.

In [142]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 
df

,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2
...,...,...,...,...,...,...,...,...
48775,2021-2022,2,42,5/29/22,None,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,5/29/22,None,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,5/29/22,None,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,5/29/22,None,CD Tenerife,FC Cartagena,None


In [143]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close() 
df_teams

,season,division,rank,team,matchday,GF,GA,GD,W,L,T,Pts
0,2021-2022,1,1.0,Real Madrid,1,4.0,1.0,3.0,1.0,0.0,0.0,3.0
1,2021-2022,1,2.0,Sevilla FC,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
2,2021-2022,1,3.0,Barcelona,1,4.0,2.0,2.0,1.0,0.0,0.0,3.0
3,2021-2022,1,4.0,Atlético Madrid,1,2.0,1.0,1.0,1.0,0.0,0.0,3.0
4,2021-2022,1,5.0,Valencia,1,1.0,0.0,1.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
97549,1928-1929,1,6.0,Athletic Madrid,18,43.0,41.0,2.0,8.0,8.0,2.0,26.0
97550,1928-1929,1,7.0,Espanyol,18,32.0,38.0,-6.0,7.0,7.0,4.0,25.0
97551,1928-1929,1,8.0,Catalunya,18,45.0,49.0,-4.0,6.0,8.0,4.0,22.0
97552,1928-1929,1,9.0,Real Unión,18,40.0,42.0,-2.0,5.0,11.0,2.0,17.0


We add a new column to `df_games`, with the result of each match: (1) home team, (X) tie or (2) visitor. If the game hasn't been played yet: `None`. 

Also, we have eliminate the columns that we consider irrelevant for or model: `date`, `time`, `score`.

In [144]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None

df_games.loc[(df_games["score"].str.split(":").str[0]) > (df_games["score"].str.split(":").str[1]), "result"] = '1'
df_games.loc[(df_games["score"].str.split(":").str[0]) == (df_games["score"].str.split(":").str[1]), "result"] = 'X'
df_games.loc[(df_games["score"].str.split(":").str[0]) < (df_games["score"].str.split(":").str[1]), "result"] = '2'

df_games.drop(["date", "time", "score"], axis=1, inplace=True)
df_games

,season,division,matchday,home_team,away_team,result
0,1928-1929,1,1,Arenas Club,Athletic Madrid,2
1,1928-1929,1,1,Espanyol,Real Unión,1
2,1928-1929,1,1,Real Madrid,Catalunya,1
3,1928-1929,1,1,Donostia,Athletic,X
4,1928-1929,1,1,Racing,Barcelona,2
...,...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,CD Tenerife,FC Cartagena,None


Now we split it in two data frames, one for the `home_team` an another for the `away_team`.

In [145]:
df_games_home = df_games[["season", "division", "matchday", "home_team", "result"]]
df_games_visitor = df_games[["season", "division", "matchday", "away_team", "result"]]

The following variables are the ones provided by the user.

In [146]:
# info for the training
season_train = "1997:2001"
# info for predict
division = 1
season = "2021-2022"
matchday = 2

We prepare the dataframes to train the model. First we select only the season from the variable `season_train`. In this example, `2000:2010` means from season `2000-2001` to `2009-2010`.

In [147]:
# Data frame of clasification
df_teams_train = df_teams

df_teams_train["season"] = df_teams_train["season"].astype(str)
df_teams_train = df_teams_train.loc[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]


In [148]:
df_games_train = df_games

df_teams_train.loc[:,"season"] = df_teams_train["season"].astype(str)
df_teams_train = df_teams_train.loc[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]

In [149]:
#función merge and clean
def merge_and_clean_home(df_games, df_teams):
    # Copiar el DataFrame para evitar modificar el original
    df_train = pd.merge(df_games.copy(), df_teams.copy(), 
                             left_on=['home_team', "season", "division", "matchday"],
                             right_on=['team', "season", "division", "matchday"])

    # Eliminar filas con valores nulos en la columna 'home_team'
    #df_train.dropna(subset=["home_team"], inplace=True)
    df_train.drop("team", axis=1, inplace=True)

    return df_train

In [150]:
#función merge and clean
def merge_and_clean_visitor(df2, df_teams):
    # Copiar el DataFrame para evitar modificar el original
    df_train = pd.merge(df2.copy(), df_teams.copy(), 
                             left_on=['away_team', "season", "division", "matchday"],
                             right_on=['team', "season", "division", "matchday"],
                             suffixes=("_home", "_away"))

    # Eliminar filas con valores nulos en la columna 'home_team'
    #df_train.dropna(subset=["away_team"], inplace=True)
    df_train.drop("team", axis=1, inplace=True)

    return df_train

In [151]:
df_train = merge_and_clean_home(df_games_train, df_teams_train)
df_train = merge_and_clean_visitor(df_train, df_teams)
df_train

,season,division,matchday,home_team,away_team,result,rank_home,GF_home,GA_home,GD_home,...,T_home,Pts_home,rank_away,GF_away,GA_away,GD_away,W_away,L_away,T_away,Pts_away
0,1997-1998,1,1,Real Madrid,Atlético Madrid,X,10.0,1.0,1.0,0.0,...,1.0,1.0,9.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
1,1997-1998,1,1,SD Compostela,Sporting Gijón,1,5.0,2.0,0.0,2.0,...,0.0,3.0,19.0,0.0,2.0,-2.0,0.0,1.0,0.0,0.0
2,1997-1998,1,1,Real Oviedo,CP Mérida,1,4.0,2.0,0.0,2.0,...,0.0,3.0,18.0,0.0,2.0,-2.0,0.0,1.0,0.0,0.0
3,1997-1998,1,1,Racing,UD Salamanca,1,8.0,1.0,0.0,1.0,...,0.0,3.0,15.0,0.0,1.0,-1.0,0.0,1.0,0.0,0.0
4,1997-1998,1,1,Barcelona,Real Sociedad,1,1.0,3.0,0.0,3.0,...,0.0,3.0,20.0,0.0,3.0,-3.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3363,2000-2001,2,42,Albacete,Uni. Las Palmas,2,5.0,46.0,40.0,6.0,...,12.0,66.0,20.0,34.0,63.0,-29.0,8.0,19.0,15.0,39.0
3364,2000-2001,2,42,Elche CF,SD Compostela,X,18.0,45.0,56.0,-11.0,...,16.0,46.0,19.0,41.0,65.0,-24.0,11.0,19.0,12.0,45.0
3365,2000-2001,2,42,Sporting Gijón,Córdoba CF,X,7.0,55.0,49.0,6.0,...,12.0,63.0,12.0,40.0,43.0,-3.0,14.0,14.0,14.0,56.0
3366,2000-2001,2,42,Real Jaén CF,Real Betis,2,10.0,37.0,40.0,-3.0,...,14.0,59.0,2.0,49.0,32.0,17.0,21.0,9.0,12.0,75.0


In [153]:
X_train = df_train[['rank_home', 'GD_home', "W_home", "Pts_home", 'rank_away', 'GD_away', "W_away", "Pts_away"]]  
y_train = df_train['result']  

In [154]:
def select_data(df, season, division, matchday):
    df_test = df[(df["season"] == season) & 
                (df["division"] == division) & 
                (df["matchday"] == matchday)]
    return df_test

In [155]:
df_teams_test = select_data(df_teams, season, division, matchday)
df_games_test = select_data(df_games, season, division, matchday)

In [156]:
df_test = merge_and_clean_home(df_games_test,df_teams_test)
df_test = merge_and_clean_visitor(df_test, df_teams_test)
df_test

,season,division,matchday,home_team,away_team,result,rank_home,GF_home,GA_home,GD_home,...,T_home,Pts_home,rank_away,GF_away,GA_away,GD_away,W_away,L_away,T_away,Pts_away
0,2021-2022,1,2,Real Betis,Cádiz CF,X,10.0,2.0,2.0,0.0,...,2.0,2.0,9.0,2.0,2.0,0.0,0.0,0.0,2.0,2.0
1,2021-2022,1,2,Alavés,RCD Mallorca,2,19.0,1.0,5.0,-4.0,...,0.0,0.0,5.0,2.0,1.0,1.0,1.0,0.0,1.0,4.0
2,2021-2022,1,2,Granada CF,Valencia,X,12.0,1.0,1.0,0.0,...,2.0,2.0,6.0,2.0,1.0,1.0,1.0,0.0,1.0,4.0
3,2021-2022,1,2,Espanyol,Villarreal,X,14.0,0.0,0.0,0.0,...,2.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
4,2021-2022,1,2,Athletic,Barcelona,X,11.0,1.0,1.0,0.0,...,2.0,2.0,4.0,5.0,3.0,2.0,1.0,0.0,1.0,4.0
5,2021-2022,1,2,Real Sociedad,Rayo Vallecano,1,7.0,3.0,4.0,-1.0,...,0.0,3.0,20.0,0.0,4.0,-4.0,0.0,2.0,0.0,0.0
6,2021-2022,1,2,Atlético Madrid,Elche CF,1,2.0,3.0,1.0,2.0,...,0.0,6.0,17.0,0.0,1.0,-1.0,0.0,1.0,1.0,1.0
7,2021-2022,1,2,Levante,Real Madrid,X,8.0,4.0,4.0,0.0,...,2.0,2.0,3.0,7.0,4.0,3.0,1.0,0.0,1.0,4.0
8,2021-2022,1,2,Getafe,Sevilla FC,2,18.0,0.0,2.0,-2.0,...,0.0,0.0,1.0,4.0,0.0,4.0,2.0,0.0,0.0,6.0
9,2021-2022,1,2,CA Osasuna,Celta de Vigo,X,13.0,0.0,0.0,0.0,...,2.0,2.0,16.0,1.0,2.0,-1.0,0.0,1.0,1.0,1.0


In [158]:
X_test = df_test[['rank_home', 'GD_home', "W_home", "Pts_home", 'rank_away', 'GD_away', "W_away", "Pts_away"]]  
y_test = df_test['result']

In [159]:
# Crear el clasificador RandomForest
rf_classifier = RandomForestClassifier()

# Definir los parámetros que deseas ajustar
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Realizar la búsqueda de cuadrícula en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el modelo con los mejores parámetros
best_rf_model = grid_search.best_estimator_

Mejores parámetros: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


In [160]:
# Evaluar el modelo en el conjunto de prueba
y_pred = best_rf_model.predict(X_test)
y_pred

array(['X', '2', 'X', 'X', 'X', '1', '1', 'X', '2', 'X'], dtype=object)

In [161]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
predictions = best_rf_model.predict_proba(X_test)

print(f"Accuracy: {accuracy}\n")
print(f"Confusion Matrix:\n{confusion_mat}\n")
print(f"Classification Report:\n{class_report}\n")
print(f"Predictions probability:\n{predictions}\n")

Accuracy: 1.0

Confusion Matrix:
[[2 0 0]
 [0 2 0]
 [0 0 6]]

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           X       1.00      1.00      1.00         6

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10


Predictions probability:
[[2.00000000e-02 0.00000000e+00 9.80000000e-01]
 [8.24742268e-04 9.65776250e-01 3.33990073e-02]
 [1.46106910e-02 4.08713157e-02 9.44517993e-01]
 [8.82352941e-03 3.52941176e-03 9.87647059e-01]
 [1.99678338e-02 6.46094109e-02 9.15422755e-01]
 [9.63462007e-01 1.49532710e-03 3.50426656e-02]
 [8.83838096e-01 2.08097051e-02 9.53521989e-02]
 [1.24224448e-01 1.65858088e-01 7.09917464e-01]
 [3.14113764e-03 9.86097629e-01 1.07612330e-02]
 [2.00000000e-02 0.00000000e+00 9.80000000e-01]]

